# 6 Deployment — Zusammenfassung & Ausblick

## 6.1 Allgemeine Erkenntnisse & Diskussion
Die Evaluation aus Notebook 5 zeigt, dass SHAP und Captum beide valide Erklärungen liefern, sich jedoch in Qualität, Robustheit und Geschwindigkeit deutlich unterscheiden. Der folgende Überblick verdichtet die wichtigsten Resultate und bildet die Grundlage für produktseitige Entscheidungen.

### 6.1.1 Hypothesenkonsolidierung
| Hypothese | Ergebnis | Evidenz (Notebook 5) |
| --- | --- | --- |
| H1: Token-Übereinstimmung | *Teilweise bestätigt* | Top-5-Überlappung Ø 0,54; FN ≈ 0,57, FP ≈ 0,46 |
| H2: Faithfulness | *Bestätigt für SHAP* | SHAP AUC-Ratio ≥ 1,5 in 73 % der Fälle; Captum 49 % |
| H3: Robustheit | *Bestätigt für SHAP, eingeschränkt für Captum* | SHAP ρ ≈ 0,75 stabil, Captum ρ ≈ 0,44 (Füllwörter kritisch) |
| H4: Rechenaufwand | *Bestätigt (Captum schneller)* | SHAP ≈ 22,9 s vs. Captum ≈ 10,0 s (≈ 2,3× schneller) |

### 6.1.2 Kernaussagen für Stakeholder
- **Qualität vs. Kosten:** SHAP liefert konsistentere und robustere Erklärungen, kostet jedoch mehr Laufzeit. Captum eignet sich für schnelle Erstanalysen.
- **Fehlerfokus:** Divergenzen treten besonders bei False Positives auf. Eine manuelle Zweitprüfung dieser Fälle ist empfehlenswert.
- **XAI-Design:** Eine hybride Strategie (Captum für Streaming, SHAP für Audits) verbindet Geschwindigkeit und Aussagekraft.

## 6.2 Überlegungen zum Deployment
Ziel ist ein reproduzierbarer, wartbarer Inferenz-Stack, der sowohl Modellvorhersagen als auch erklärende Artefakte bereitstellt.

### 6.2.1 Deployment-Ziele
- **Hauptziel:** Bereitstellung eines Sentiment-Services mit optionalen XAI-Erklärungen (SHAP für Audits, Captum für Online-Erklärbarkeit).
- **Sekundärziel:** Sicherstellung, dass Ergebnisse aus Notebook 5 reproduzierbar bleiben (Versionierung von Code, Daten, Modellen).

### 6.2.2 Technische Anforderungen
- **Artefakte:** Feinabgestimmtes `vinai/bertweet-base`, Tokenizer, PyTorch-Lightning-Checkpoint, Konfigurationsdateien.
- **Laufzeitumgebung:** Python 3.10+, PyTorch ≥ 2.0, Transformers ≥ 4.57, Captum, SHAP, PyArrow (für Caching), optional ONNX Runtime für Beschleunigung.
- **Hardware:** CPU-Only Betrieb möglich (≈ 100 ms inference pro Tweet), GPU/MPS zur Beschleunigung von Batches und SHAP-Analysen empfehlenswert.
- **Konfigurierbarkeit:** Environment Variablen für Modellpfade, Batchgrößen, Wahl der Erklärmethode.

### 6.2.3 Serving-Architektur
1. **Preprocessing-Service:** Repliziert die Cleaning-Logik aus Notebook 3 (Regex, Normalisierung).
2. **Inference-Service:** Lädt das fine-tuned Modell, bietet REST/gRPC-Endpunkte für Sentiment-Predictions.
3. **Explainability-Service:**
   - *Captum-Pfad:* Schnell, liefert Top-Tokens via LayerIntegratedGradients.
   - *SHAP-Pfad:* Asynchroner Job (Celery/Kafka) für tiefgehende Analysen; Ergebnisse werden gecached.
4. **Storage & Caching:**
   - Redis / SQLite für kurzfristige Ergebnisse.
   - Langfristige Speicherung in S3/Parquet (bereits vorbereitet in Notebook 5).

### 6.2.4 Betriebsprozesse & Überwachung
- **Monitoring:**
  - Metriken: Latenz (p95), Fehlerraten, Anteil der Erklärungsanfragen, Drift-Indikatoren auf Token-Ebene.
  - Logging: Speicherung von (Text, Prediction, Explanation-ID) für Audits (DSGVO-konform).
- **Alerting:** Schwellen für Latenz > 500 ms oder wiederholte Ausfälle beim SHAP-Worker.
- **Governance:** Versionierung über DVC/MLflow; jedes Deployment enthält Modell-, Daten- und Evaluations-Hash.

### 6.2.5 Produktionsreife Checkliste
1. **Reproduzierbares Build:** Dockerfile mit Mehrstufen-Build, automatisierte Tests (PyTest, Smoke Tests).
2. **CI/CD-Pipeline:** GitHub Actions oder Azure DevOps; Stages: Lint → Unit Tests → Modell-Drift-Check → Deployment (staging → prod).
3. **Security:** Secrets-Management, Rate Limiting, Eingabevalidierung.
4. **UX/API:** Konsistentes Schema (JSON) für Antwort inkl. Scores, Token-Attributionslisten, optionaler Visualisierungshyperlink.
5. **Rollbacks:** Canary-Deployments, automatisiertes Zurücksetzen bei Regressionen.

## 6.3 Einschränkungen & Nächste Schritte
Eine reflektierte Betrachtung der Projektgrenzen hilft, Risiken beim Deployment zu adressieren und die Roadmap für weitere Iterationen festzulegen.

### 6.3.1 Bekannte Einschränkungen
- **Datenbasis:** Fokus auf englische Tweets aus 2009; Generalisierbarkeit auf aktuelle Plattformen oder andere Sprachen nicht geprüft.
- **Attributionsstreuung:** Captum-Sensitivität bei Perturbationen deutet auf mögliche Instabilität im Live-Betrieb.
- **Compute-Kosten:** SHAP-Aufwand skaliert schlecht; Produktivbetrieb benötigt dedizierte Ressourcen oder Sampling-Strategien.
- **Bias-Risiko:** Keine Fairness- oder Bias-Audits durchgeführt; vor Deployment erforderlich.

### 6.3.2 Empfohlene nächste Schritte
1. **Erweiterte Evaluation:** Größere Stichprobe (> 200 Tweets), zusätzliche Perturbationen (Code-Switching, Emojis).
2. **User Research:** Interviews mit Analyst:innen zur Verständlichkeit der Erklärungen, Anpassung des UI basierend auf Feedback.
3. **Hybrid-Explainability:** Implementierung eines zweistufigen Systems (Captum online, SHAP offline) und A/B-Test gegen rein Captum-basierten Flow.
4. **Model Monitoring MVP:** Aufbau eines Monitoring-Dashboards (Grafana/Prometheus) mit Echtzeit-Alarmierung.
5. **Compliance-Check:** Datenschutz-Freigabe, Dokumentation für Audit (Modellkarten, Datenblätter).

## 6.4 Roadmap zur Produktionsreife
- **Kurzfristig (0–1 Monat):** Dockerisieren, CI/CD aufsetzen, Captum-Only Service ausrollen, Monitoring-Basics implementieren.
- **Mittelfristig (1–3 Monate):** SHAP-Offloading-Service etablieren, UI-Komponenten bauen, Fairness-Checks durchführen.
- **Langfristig (>3 Monate):** Erweiterung auf Multilingualität, kontinuierliches Retraining mit Feedback-Schleifen, Integration in CRM/Support-Systeme.